In [ ]:
!pip install gradio

In [6]:
import gradio as gr
import requests
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
import numpy as np

def fetch_species_data(lat, lon, radius_km=1):
    """
    Fetch species data from iNaturalist API for given coordinates
    """
    # Convert radius to degrees (approximate)
    radius_deg = radius_km / 111.32  # 1 degree is approximately 111.32 km

    # Calculate bounding box
    bbox = f"{lon-radius_deg},{lat-radius_deg},{lon+radius_deg},{lat+radius_deg}"

    # Get observations from the last year
    one_year_ago = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

    # iNaturalist API endpoint
    url = "https://api.inaturalist.org/v1/observations"

    params = {
        "bbox": bbox,
        "per_page": 200,  # Maximum allowed per page
        "order": "desc",
        "order_by": "observed_on",
        "d1": one_year_ago,
        "verifiable": "true",
        "quality_grade": "research",  # Only research grade observations
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract relevant information
        species_data = []
        for obs in data['results']:
            if obs.get('taxon'):
                species_data.append({
                    'Scientific Name': obs['taxon'].get('name', 'Unknown'),
                    'Common Name': obs['taxon'].get('preferred_common_name', 'Unknown'),
                    'Kingdom': obs['taxon'].get('kingdom_name', 'Unknown'),
                    'Family': obs['taxon'].get('family_name', 'Unknown'),
                    'Observations Count': obs['taxon'].get('observations_count', 0),
                    'Last Observed': obs.get('observed_on', 'Unknown'),
                })

        # Convert to DataFrame and remove duplicates
        df = pd.DataFrame(species_data).drop_duplicates(subset=['Scientific Name'])
        df = df.sort_values('Observations Count', ascending=False)

        if df.empty:
            return pd.DataFrame({'Message': ['No species found in this location']})

        return df
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({'Error': [f'API request failed: {str(e)}']})

def create_map(lat=40.7128, lon=-74.0060, radius=1):
    """Create map with click event handling"""
    fig = go.Figure(go.Scattermapbox(
        lat=[lat],
        lon=[lon],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=12,
            color='red'
        ),
        hoverinfo="text",
        hovertemplate='Selected Location<br>Lat: %{lat}<br>Lon: %{lon}'
    ))

    # Add circle to show search radius
    circle_points = []
    for angle in range(0, 361, 10):
        km_lat = radius / 111.32  # convert km to degrees latitude
        km_lon = radius / (111.32 * abs(np.cos(np.radians(lat))))  # adjust for longitude
        lat_point = lat + km_lat * np.cos(np.radians(angle))
        lon_point = lon + km_lon * np.sin(np.radians(angle))
        circle_points.append((lat_point, lon_point))

    circle_lats, circle_lons = zip(*circle_points)
    fig.add_trace(go.Scattermapbox(
        lat=list(circle_lats),
        lon=list(circle_lons),
        mode='lines',
        line=dict(color='red', width=2),
        hoverinfo='skip'
    ))

    fig.update_layout(
        mapbox_style="open-street-map",
        hovermode='closest',
        mapbox=dict(
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=lat,
                lon=lon
            ),
            pitch=0,
            zoom=11
        ),
    )

    # Add click event handling
    fig.update_layout(
        clickmode='event',
        margin=dict(l=0, r=0, t=0, b=0)
    )

    return fig

def handle_click(click_data, radius):
    """Process map click and return updated results"""
    if not click_data or 'points' not in click_data or not click_data['points']:
        return create_map(), pd.DataFrame({'Message': ['Please click on the map']})

    point = click_data['points'][0]
    lat, lon = point['lat'], point['lon']

    # Update map with new marker
    updated_map = create_map(lat, lon, radius)

    # Get species data
    species_df = fetch_species_data(lat, lon, radius)

    return updated_map, species_df

# Create Gradio interface
with gr.Blocks() as inat_app:
    gr.Markdown("# iNaturalist Species Finder")
    gr.Markdown("Click anywhere on the map to see species observed within the specified radius in the last year")

    with gr.Row():
        with gr.Column():
            radius = gr.Slider(
                minimum=0.1,
                maximum=10,
                value=1,
                step=0.1,
                label="Search Radius (km)"
            )
            map_plot = gr.Plot(value=create_map())
            click_data = gr.JSON(value={}, visible=False)

    with gr.Row():
        species_table = gr.Dataframe(
            headers=[
                "Scientific Name",
                "Common Name",
                "Kingdom",
                "Family",
                "Observations Count",
                "Last Observed"
            ],
            label="Species Found"
        )

    # Handle map click events through a hidden JSON component
    map_plot.change(
        fn=lambda x: x,
        inputs=map_plot,
        outputs=click_data
    )

    click_data.change(
        fn=handle_click,
        inputs=[click_data, radius],
        outputs=[map_plot, species_table]
    )

    # Update map when radius changes
    radius.change(
        fn=lambda r, cd: handle_click(cd, r) if cd and 'points' in cd else (create_map(), pd.DataFrame()),
        inputs=[radius, click_data],
        outputs=[map_plot, species_table]
    )

if __name__ == "__main__":
    inat_app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8f05697f510707c7cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
